In [ ]:
import pandas as pd
from IPython.core.pylabtools import figsize
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(42)


In [ ]:
df = pd.read_csv('./fmnist_small.csv')

In [ ]:
df.head()

In [ ]:
fig,axes = plt.subplots(4,4,figsize=(10,10))
fig.suptitle("16 images")
for i,ax in enumerate(axes.flat):
    img = df.iloc[i,1:].values.reshape(28,28)
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(f"label {df.iloc[i,0]}")

plt.tight_layout(rect=[0,0,1,1])
plt.show()

In [ ]:
x = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
xtrain = xtrain/255.0
xtest = xtest/255.0

In [ ]:
class customdataset(Dataset):
    def __init__(self,features,labels):
        self.features = torch.tensor(features,dtype=torch.float32)
        self.labels = torch.tensor(labels,dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx],self.labels[idx]


In [ ]:
traindataset = customdataset(xtrain,ytrain)

In [ ]:
traindataset[0]

In [ ]:
testdataset = customdataset(xtest,ytest)

In [ ]:
trainloader = DataLoader(traindataset,batch_size=32,shuffle=True)
testloader = DataLoader(testdataset,batch_size=32,shuffle=False)

In [ ]:
class MyNN(nn.Module):
    def __init__(self,numfeatures):
        super().__init__()
        self.model = nn.Sequential(nn.Linear(numfeatures,128),nn.ReLU(),nn.Linear(128,64),nn.ReLU(),nn.Linear(64,10))

    def forward(self,x):
        return self.model(x)


In [ ]:
epochs = 100
lr = 0.1

In [ ]:
model = MyNN(xtrain.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=lr)

In [ ]:
for epoch in range(epochs):
    for batchfeatures,batchlabels in trainloader:
        output = model(batchfeatures)
        loss = criterion(output,batchlabels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"epoch {epoch+1}, loss: {loss.item():.4f}")


In [ ]:
model.eval()

In [ ]:
total =0
correct = 0
with torch.no_grad():
    for batchfeatures,batchlabels in testloader:
        output = model(batchfeatures)
        _, predicted = torch.max(output.data,1)
        total += batchlabels.shape[0]
        correct = correct + predicted.eq(batchlabels).sum().item()
print(correct/total)